# Métodos Basados en Densidad


## DBSCAN, OPTICS y HDBSCAN

---
### Objetivos del Notebook

1. Comprender el paradigma de clustering basado en densidad y su diferencia con métodos particionales
2. Dominar los conceptos de epsilon-vecindad, puntos núcleo, borde y ruido en DBSCAN
3. Aplicar técnicas de selección de parámetros eps y min_samples mediante k-distance graph
4. Conocer las extensiones OPTICS y HDBSCAN para superar limitaciones de DBSCAN
5. Comparar el rendimiento de métodos basados en densidad con K-Means

---

## 1. Configuración del Entorno

Importamos las bibliotecas necesarias para el desarrollo del módulo.

In [ ]:
# Instalación de HDBSCAN (si no está disponible)
!pip install hdbscan -q

In [ ]:
# Bibliotecas fundamentales
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import warnings
warnings.filterwarnings('ignore')

# Scikit-learn: clustering y métricas
from sklearn.cluster import DBSCAN, OPTICS, KMeans
from sklearn.datasets import make_blobs, make_moons, make_circles
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score, adjusted_rand_score

# HDBSCAN
import hdbscan

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 13
plt.rcParams['axes.labelsize'] = 11

# Reproducibilidad
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

print("Entorno configurado correctamente.")

## 2. Motivación: Limitaciones de K-Means

Los métodos particionales como K-Means asumen clusters de forma convexa (esférica). Demostraremos sus limitaciones con datos de forma no convexa.

In [ ]:
# Generar datasets con diferentes estructuras
n_samples = 500

# Dataset 1: Lunas (no convexo)
X_moons, y_moons = make_moons(n_samples=n_samples, noise=0.05, random_state=RANDOM_STATE)

# Dataset 2: Círculos concéntricos (no convexo)
X_circles, y_circles = make_circles(n_samples=n_samples, noise=0.05, factor=0.5, random_state=RANDOM_STATE)

# Dataset 3: Blobs con ruido (convexo con outliers)
X_blobs, y_blobs = make_blobs(n_samples=400, centers=3, cluster_std=0.6, random_state=RANDOM_STATE)
# Añadir outliers
outliers = np.random.uniform(low=-10, high=10, size=(50, 2))
X_blobs_noisy = np.vstack([X_blobs, outliers])
y_blobs_noisy = np.hstack([y_blobs, [-1]*50])

# Visualización de los datasets
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].scatter(X_moons[:, 0], X_moons[:, 1], c=y_moons, cmap='viridis', edgecolors='w', s=40)
axes[0].set_title('Lunas (no convexo)')
axes[0].set_xlabel('Característica 1')
axes[0].set_ylabel('Característica 2')

axes[1].scatter(X_circles[:, 0], X_circles[:, 1], c=y_circles, cmap='viridis', edgecolors='w', s=40)
axes[1].set_title('Círculos concéntricos (no convexo)')
axes[1].set_xlabel('Característica 1')
axes[1].set_ylabel('Característica 2')

colors = ['#3498db' if y >= 0 else '#e74c3c' for y in y_blobs_noisy]
axes[2].scatter(X_blobs_noisy[:, 0], X_blobs_noisy[:, 1], c=colors, edgecolors='w', s=40)
axes[2].set_title('Blobs con outliers')
axes[2].set_xlabel('Característica 1')
axes[2].set_ylabel('Característica 2')

plt.tight_layout()
plt.show()

In [ ]:
# Aplicar K-Means a los datasets
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

datasets = [
    (X_moons, y_moons, 'Lunas'),
    (X_circles, y_circles, 'Círculos'),
    (X_blobs_noisy, y_blobs_noisy, 'Blobs con outliers')
]

for idx, (X, y_true, titulo) in enumerate(datasets):
    ax = axes[idx]

    # K-Means con k=2 para lunas/círculos, k=3 para blobs
    k = 2 if idx < 2 else 3
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels = kmeans.fit_predict(X)

    # Calcular ARI (excluyendo outliers para blobs)
    if idx < 2:
        ari = adjusted_rand_score(y_true, labels)
    else:
        mask = y_true >= 0
        ari = adjusted_rand_score(y_true[mask], labels[mask])

    ax.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', edgecolors='w', s=40)
    ax.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1],
               c='red', marker='X', s=200, edgecolors='w', linewidths=2)
    ax.set_title(f'{titulo} - K-Means (ARI: {ari:.3f})')
    ax.set_xlabel('Característica 1')
    ax.set_ylabel('Característica 2')

plt.suptitle('Limitaciones de K-Means con estructuras no convexas', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("K-Means falla en detectar clusters no convexos y es sensible a outliers.")

## 3. DBSCAN: Fundamentos

### 3.1 Conceptos Básicos

DBSCAN (Density-Based Spatial Clustering of Applications with Noise) define clusters como regiones de alta densidad separadas por regiones de baja densidad.

In [ ]:
# Demostración de conceptos: epsilon-vecindad y tipos de puntos
np.random.seed(RANDOM_STATE)

# Crear un pequeño dataset para ilustración
X_demo = np.array([
    [1, 1], [1.2, 1.1], [0.9, 1.3], [1.1, 0.8],  # Cluster denso
    [1.3, 1.2], [0.8, 1.0], [1.0, 1.4],
    [4, 4], [4.2, 4.1], [3.9, 4.2],              # Cluster pequeño
    [2.5, 2.5],                                   # Punto borde potencial
    [6, 1]                                        # Outlier
])

eps = 0.4
min_samples = 4

# Calcular vecindades
nn = NearestNeighbors(radius=eps)
nn.fit(X_demo)
vecindades = nn.radius_neighbors(X_demo, return_distance=False)

# Clasificar puntos
n_vecinos = [len(v) for v in vecindades]
tipos = []
for i, n in enumerate(n_vecinos):
    if n >= min_samples:
        tipos.append('Núcleo')
    else:
        # Verificar si está en la vecindad de un punto núcleo
        es_borde = False
        for j, vecindad in enumerate(vecindades):
            if len(vecindad) >= min_samples and i in vecindad:
                es_borde = True
                break
        tipos.append('Borde' if es_borde else 'Ruido')

# Visualización
fig, ax = plt.subplots(figsize=(10, 8))

colores_tipo = {'Núcleo': '#2ecc71', 'Borde': '#f39c12', 'Ruido': '#e74c3c'}
marcadores = {'Núcleo': 'o', 'Borde': 's', 'Ruido': 'x'}

for tipo in ['Núcleo', 'Borde', 'Ruido']:
    mask = [t == tipo for t in tipos]
    puntos = X_demo[mask]
    ax.scatter(puntos[:, 0], puntos[:, 1], c=colores_tipo[tipo],
               marker=marcadores[tipo], s=150, label=f'{tipo} ({sum(mask)})',
               edgecolors='w', linewidths=2)

# Dibujar epsilon-vecindades para puntos núcleo
for i, (punto, tipo) in enumerate(zip(X_demo, tipos)):
    if tipo == 'Núcleo':
        circulo = plt.Circle(punto, eps, fill=False, color='#2ecc71',
                            linestyle='--', alpha=0.5, linewidth=2)
        ax.add_patch(circulo)

ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')
ax.set_title(f'Tipos de puntos en DBSCAN (eps={eps}, min_samples={min_samples})')
ax.legend(loc='upper right')
ax.set_aspect('equal')
ax.set_xlim(-0.5, 7.5)
ax.set_ylim(-0.5, 5.5)

plt.tight_layout()
plt.show()

print("Clasificación de puntos:")
for i, (punto, tipo, n) in enumerate(zip(X_demo, tipos, n_vecinos)):
    print(f"  Punto {i}: {punto} -> {tipo} (vecinos en eps: {n})")

### 3.2 Implementación de DBSCAN con Scikit-learn

In [ ]:
# Aplicar DBSCAN a los datasets problemáticos para K-Means
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

# Parámetros ajustados para cada dataset
params = [
    {'eps': 0.1, 'min_samples': 5},   # Lunas
    {'eps': 0.13, 'min_samples': 5},   # Círculos
    {'eps': 0.8, 'min_samples': 5}    # Blobs con outliers
]

for idx, (X, y_true, titulo) in enumerate(datasets):
    # K-Means (fila superior)
    ax_km = axes[0, idx]
    k = 2 if idx < 2 else 3
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
    labels_km = kmeans.fit_predict(X)

    if idx < 2:
        ari_km = adjusted_rand_score(y_true, labels_km)
    else:
        mask = y_true >= 0
        ari_km = adjusted_rand_score(y_true[mask], labels_km[mask])

    ax_km.scatter(X[:, 0], X[:, 1], c=labels_km, cmap='viridis', edgecolors='w', s=30)
    ax_km.set_title(f'K-Means (ARI: {ari_km:.3f})')
    if idx == 0:
        ax_km.set_ylabel('K-Means', fontsize=12, fontweight='bold')

    # DBSCAN (fila inferior)
    ax_db = axes[1, idx]
    dbscan = DBSCAN(**params[idx])
    labels_db = dbscan.fit_predict(X)

    # Calcular métricas (excluyendo ruido)
    mask_no_ruido = labels_db >= 0
    if mask_no_ruido.sum() > 0 and len(np.unique(labels_db[mask_no_ruido])) > 1:
        if idx < 2:
            ari_db = adjusted_rand_score(y_true[mask_no_ruido], labels_db[mask_no_ruido])
        else:
            mask_valid = (y_true >= 0) & (labels_db >= 0)
            ari_db = adjusted_rand_score(y_true[mask_valid], labels_db[mask_valid])
    else:
        ari_db = 0

    n_clusters = len(set(labels_db)) - (1 if -1 in labels_db else 0)
    n_ruido = (labels_db == -1).sum()

    # Colorear: clusters con colores, ruido en gris
    colors = plt.cm.viridis(np.linspace(0, 1, max(n_clusters, 1)))
    for cluster_id in range(n_clusters):
        mask = labels_db == cluster_id
        ax_db.scatter(X[mask, 0], X[mask, 1], c=[colors[cluster_id]], edgecolors='w', s=30)

    mask_ruido = labels_db == -1
    if mask_ruido.sum() > 0:
        ax_db.scatter(X[mask_ruido, 0], X[mask_ruido, 1], c='gray', marker='x', s=30, alpha=0.5)

    ax_db.set_title(f'DBSCAN (ARI: {ari_db:.3f}, clusters: {n_clusters}, ruido: {n_ruido})')
    ax_db.set_xlabel(titulo)
    if idx == 0:
        ax_db.set_ylabel('DBSCAN', fontsize=12, fontweight='bold')

plt.suptitle('Comparación K-Means vs DBSCAN', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.3 Selección de Parámetros: Método k-Distance Graph

El método del k-distance graph permite estimar un valor apropiado de eps.

In [ ]:
def plot_k_distance(X, k=5, ax=None):
    """
    Genera el gráfico de k-distancias para seleccionar eps.

    Parámetros:
    -----------
    X : array-like
        Datos de entrada.
    k : int
        Número de vecinos (típicamente min_samples).
    ax : matplotlib axis
        Eje para graficar.

    Retorna:
    --------
    k_distances : array
        Distancias al k-ésimo vecino ordenadas.
    """
    nn = NearestNeighbors(n_neighbors=k)
    nn.fit(X)
    distances, _ = nn.kneighbors(X)

    # Distancia al k-ésimo vecino
    k_distances = np.sort(distances[:, k-1])

    if ax is not None:
        ax.plot(range(len(k_distances)), k_distances, 'b-', linewidth=1.5)
        ax.set_xlabel('Puntos ordenados')
        ax.set_ylabel(f'Distancia al {k}-ésimo vecino')
        ax.set_title(f'k-Distance Graph (k={k})')
        ax.grid(True, alpha=0.3)

    return k_distances

# Aplicar a los datasets
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
k = 5  # min_samples típico

for idx, (X, _, titulo) in enumerate(datasets):
    ax = axes[idx]
    k_dist = plot_k_distance(X, k=k, ax=ax)
    ax.set_title(f'{titulo}\n(k={k})')

    # Marcar zona del codo
    if idx == 0:  # Lunas
        ax.axhline(y=0.1, color='red', linestyle='--', label='eps sugerido: 0.1')
    elif idx == 1:  # Círculos
        ax.axhline(y=0.13, color='red', linestyle='--', label='eps sugerido: 0.13')
    else:  # Blobs con outliers
        ax.axhline(y=0.8, color='red', linestyle='--', label='eps sugerido: 0.8')
    ax.legend()

plt.suptitle('Método k-Distance para selección de eps', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("El punto de inflexión (codo) en la curva indica el valor óptimo de eps.")
print("Puntos debajo del codo pertenecen a clusters; puntos encima son potencial ruido.")

### 3.4 Sensibilidad a los Parámetros

In [ ]:
# Demostrar el efecto de eps en DBSCAN
eps_values = [0.1, 0.2, 0.3, 0.5]

fig, axes = plt.subplots(2, 4, figsize=(18, 9))

for row, (X, y_true, titulo) in enumerate([(X_moons, y_moons, 'Lunas'),
                                            (X_circles, y_circles, 'Círculos')]):
    for col, eps in enumerate(eps_values):
        ax = axes[row, col]

        dbscan = DBSCAN(eps=eps, min_samples=5)
        labels = dbscan.fit_predict(X)

        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_ruido = (labels == -1).sum()

        # Calcular ARI excluyendo ruido
        mask = labels >= 0
        if mask.sum() > 0 and n_clusters > 1:
            ari = adjusted_rand_score(y_true[mask], labels[mask])
        else:
            ari = 0

        # Visualizar
        scatter = ax.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis',
                            edgecolors='w', s=30, alpha=0.7)
        ax.set_title(f'eps={eps}\nclusters={n_clusters}, ruido={n_ruido}\nARI={ari:.3f}')

        if col == 0:
            ax.set_ylabel(titulo, fontsize=12, fontweight='bold')

plt.suptitle('Efecto del parámetro eps en DBSCAN (min_samples=5)',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### 3.5 Limitación: Densidad Variable

DBSCAN tiene dificultades cuando los clusters tienen densidades muy diferentes.

In [ ]:
# Crear dataset con clusters de diferente densidad
np.random.seed(RANDOM_STATE)

# Cluster denso
cluster_denso = np.random.randn(200, 2) * 0.3 + np.array([0, 0])

# Cluster disperso
cluster_disperso = np.random.randn(200, 2) * 1.5 + np.array([5, 5])

X_variable = np.vstack([cluster_denso, cluster_disperso])
y_variable = np.array([0]*200 + [1]*200)

# Probar diferentes valores de eps
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

eps_values = [0.3, 0.6, 1.0, 1.5]

axes[0].scatter(X_variable[:, 0], X_variable[:, 1], c=y_variable, cmap='viridis',
                edgecolors='w', s=40)
axes[0].set_title('Ground Truth')
axes[0].set_xlabel('Característica 1')
axes[0].set_ylabel('Característica 2')

for idx, eps in enumerate(eps_values[:-1]):
    ax = axes[idx + 1]

    dbscan = DBSCAN(eps=eps, min_samples=5)
    labels = dbscan.fit_predict(X_variable)

    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_ruido = (labels == -1).sum()

    ax.scatter(X_variable[:, 0], X_variable[:, 1], c=labels, cmap='viridis',
               edgecolors='w', s=40)
    ax.set_title(f'DBSCAN eps={eps}\nclusters={n_clusters}, ruido={n_ruido}')
    ax.set_xlabel('Característica 1')

plt.suptitle('Limitación de DBSCAN: Clusters con densidad variable',
             fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

print("Con eps pequeño: el cluster disperso se fragmenta o se marca como ruido.")
print("Con eps grande: los clusters se fusionan o el denso pierde definición.")
print("\nEsta limitación motiva el desarrollo de OPTICS y HDBSCAN.")

## 4. OPTICS: Ordering Points To Identify Clustering Structure

OPTICS resuelve el problema de la densidad variable generando un ordenamiento de los datos que permite extraer clusters a diferentes niveles de densidad.

In [ ]:
# Aplicar OPTICS al dataset de densidad variable
optics = OPTICS(min_samples=10, xi=0.05, min_cluster_size=0.05)
labels_optics = optics.fit_predict(X_variable)

# Obtener reachability distances
reachability = optics.reachability_
ordering = optics.ordering_

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Reachability plot
ax = axes[0]
colors = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6', '#f39c12']
for i, (reach, label) in enumerate(zip(reachability[ordering], labels_optics[ordering])):
    if label == -1:
        color = 'gray'
    else:
        color = colors[label % len(colors)]
    ax.bar(i, reach, width=1, color=color, alpha=0.7)

ax.set_xlabel('Orden de procesamiento')
ax.set_ylabel('Reachability distance')
ax.set_title('Diagrama de Alcanzabilidad (OPTICS)')

# Clusters detectados por OPTICS
ax = axes[1]
n_clusters_optics = len(set(labels_optics)) - (1 if -1 in labels_optics else 0)
ax.scatter(X_variable[:, 0], X_variable[:, 1], c=labels_optics, cmap='viridis',
           edgecolors='w', s=40)
ax.set_title(f'OPTICS\nclusters={n_clusters_optics}')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# Ground truth para comparación
ax = axes[2]
ax.scatter(X_variable[:, 0], X_variable[:, 1], c=y_variable, cmap='viridis',
           edgecolors='w', s=40)
ax.set_title('Ground Truth')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

plt.tight_layout()
plt.show()

print("El diagrama de alcanzabilidad muestra la estructura de densidad:")
print("- Los valles representan clusters (regiones densas).")
print("- Los picos representan fronteras entre clusters o ruido.")

In [ ]:
# Comparación DBSCAN vs OPTICS en datasets no convexos
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

for col, (X, y_true, titulo) in enumerate(datasets):
    # DBSCAN
    ax = axes[0, col]
    dbscan = DBSCAN(eps=params[col]['eps'], min_samples=params[col]['min_samples'])
    labels_db = dbscan.fit_predict(X)

    n_clusters_db = len(set(labels_db)) - (1 if -1 in labels_db else 0)
    ax.scatter(X[:, 0], X[:, 1], c=labels_db, cmap='viridis', edgecolors='w', s=30)
    ax.set_title(f'DBSCAN (clusters={n_clusters_db})')
    if col == 0:
        ax.set_ylabel('DBSCAN', fontsize=12, fontweight='bold')

    # OPTICS
    ax = axes[1, col]
    optics = OPTICS(min_samples=5, xi=0.05, min_cluster_size=0.05)
    labels_op = optics.fit_predict(X)

    n_clusters_op = len(set(labels_op)) - (1 if -1 in labels_op else 0)
    ax.scatter(X[:, 0], X[:, 1], c=labels_op, cmap='viridis', edgecolors='w', s=30)
    ax.set_title(f'OPTICS (clusters={n_clusters_op})')
    ax.set_xlabel(titulo)
    if col == 0:
        ax.set_ylabel('OPTICS', fontsize=12, fontweight='bold')

plt.suptitle('Comparación DBSCAN vs OPTICS', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 5. HDBSCAN: Hierarchical DBSCAN

HDBSCAN combina las ventajas de DBSCAN con clustering jerárquico, permitiendo detectar clusters de densidad variable sin necesidad de especificar eps.

In [ ]:
# Aplicar HDBSCAN al dataset de densidad variable
clusterer = hdbscan.HDBSCAN(min_cluster_size=15, min_samples=5)
labels_hdb = clusterer.fit_predict(X_variable)

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Ground truth
ax = axes[0]
ax.scatter(X_variable[:, 0], X_variable[:, 1], c=y_variable, cmap='viridis',
           edgecolors='w', s=50)
ax.set_title('Ground Truth (2 clusters)')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# DBSCAN con eps "óptimo"
ax = axes[1]
dbscan = DBSCAN(eps=0.6, min_samples=5)
labels_db = dbscan.fit_predict(X_variable)
n_clusters_db = len(set(labels_db)) - (1 if -1 in labels_db else 0)
n_ruido_db = (labels_db == -1).sum()
ax.scatter(X_variable[:, 0], X_variable[:, 1], c=labels_db, cmap='viridis',
           edgecolors='w', s=50)
ax.set_title(f'DBSCAN (eps=0.6)\nclusters={n_clusters_db}, ruido={n_ruido_db}')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# HDBSCAN
ax = axes[2]
n_clusters_hdb = len(set(labels_hdb)) - (1 if -1 in labels_hdb else 0)
n_ruido_hdb = (labels_hdb == -1).sum()
ax.scatter(X_variable[:, 0], X_variable[:, 1], c=labels_hdb, cmap='viridis',
           edgecolors='w', s=50)
ax.set_title(f'HDBSCAN\nclusters={n_clusters_hdb}, ruido={n_ruido_hdb}')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

plt.suptitle('HDBSCAN vs DBSCAN en datos con densidad variable',
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Características adicionales de HDBSCAN: probabilidades y árbol condensado
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Probabilidades de pertenencia
ax = axes[0]
probabilities = clusterer.probabilities_
scatter = ax.scatter(X_variable[:, 0], X_variable[:, 1], c=probabilities,
                     cmap='viridis', edgecolors='w', s=50)
plt.colorbar(scatter, ax=ax, label='Probabilidad de pertenencia')
ax.set_title('Probabilidades de pertenencia (HDBSCAN)')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

# Outlier scores
ax = axes[1]
outlier_scores = clusterer.outlier_scores_
scatter = ax.scatter(X_variable[:, 0], X_variable[:, 1], c=outlier_scores,
                     cmap='Reds', edgecolors='w', s=50)
plt.colorbar(scatter, ax=ax, label='Outlier score')
ax.set_title('Outlier Scores (HDBSCAN)')
ax.set_xlabel('Característica 1')
ax.set_ylabel('Característica 2')

plt.tight_layout()
plt.show()

print("HDBSCAN proporciona información adicional:")
print("- Probabilidad de pertenencia: confianza en la asignación de cada punto.")
print("- Outlier scores: qué tan anómalo es cada punto (valores altos = más anómalo).")

In [ ]:
# Árbol condensado de HDBSCAN
fig, ax = plt.subplots(figsize=(12, 6))
clusterer.condensed_tree_.plot(select_clusters=True, selection_palette=['#3498db', '#e74c3c', '#2ecc71'])
plt.title('Árbol Condensado (HDBSCAN)')
plt.xlabel('Tamaño del cluster')
plt.ylabel('Lambda (1/distancia)')
plt.tight_layout()
plt.show()

print("El árbol condensado muestra la jerarquía de clusters.")
print("Las regiones coloreadas son los clusters seleccionados como más estables.")

## 6. Comparación Integral de Métodos

Comparamos sistemáticamente K-Means, DBSCAN, OPTICS y HDBSCAN.

In [ ]:
# Comparación completa en todos los datasets
fig, axes = plt.subplots(4, 4, figsize=(18, 16))

all_datasets = [
    (X_moons, y_moons, 'Lunas'),
    (X_circles, y_circles, 'Círculos'),
    (X_blobs_noisy, y_blobs_noisy, 'Blobs+Outliers'),
    (X_variable, y_variable, 'Densidad Variable')
]

metodos = ['Ground Truth', 'K-Means', 'DBSCAN', 'HDBSCAN']

for row, (X, y_true, titulo) in enumerate(all_datasets):
    for col, metodo in enumerate(metodos):
        ax = axes[row, col]

        if metodo == 'Ground Truth':
            labels = y_true
            info = ''
        elif metodo == 'K-Means':
            k = 2 if row < 2 else 3 if row == 2 else 2
            model = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
            labels = model.fit_predict(X)
            mask = y_true >= 0 if row == 2 else np.ones(len(y_true), dtype=bool)
            ari = adjusted_rand_score(y_true[mask], labels[mask])
            info = f'ARI={ari:.2f}'
        elif metodo == 'DBSCAN':
            if row == 0:
                eps_val = 0.2
            elif row == 1:
                eps_val = 0.2
            elif row == 2:
                eps_val = 0.8
            else:
                eps_val = 0.6
            model = DBSCAN(eps=eps_val, min_samples=5)
            labels = model.fit_predict(X)
            mask = (labels >= 0) & (y_true >= 0) if row == 2 else labels >= 0
            if mask.sum() > 0:
                ari = adjusted_rand_score(y_true[mask], labels[mask])
            else:
                ari = 0
            n_ruido = (labels == -1).sum()
            info = f'ARI={ari:.2f}, ruido={n_ruido}'
        else:  # HDBSCAN
            model = hdbscan.HDBSCAN(min_cluster_size=15, min_samples=5)
            labels = model.fit_predict(X)
            mask = (labels >= 0) & (y_true >= 0) if row == 2 else labels >= 0
            if mask.sum() > 0:
                ari = adjusted_rand_score(y_true[mask], labels[mask])
            else:
                ari = 0
            n_ruido = (labels == -1).sum()
            info = f'ARI={ari:.2f}, ruido={n_ruido}'

        ax.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', edgecolors='w', s=25, alpha=0.7)
        ax.set_title(f'{metodo}\n{info}' if info else metodo, fontsize=10)

        if col == 0:
            ax.set_ylabel(titulo, fontsize=11, fontweight='bold')
        if row == 3:
            ax.set_xlabel(metodo, fontsize=10)

plt.suptitle('Comparación de Algoritmos de Clustering', fontsize=14, fontweight='bold', y=1.01)
plt.tight_layout()
plt.show()

## 7. Caso Práctico: Detección de Anomalías en Datos Geoespaciales

Aplicamos los métodos basados en densidad a un problema de detección de patrones y anomalías en ubicaciones geográficas.

In [ ]:
# Simular datos de ubicaciones (ej: transacciones bancarias)
np.random.seed(RANDOM_STATE)

# Centros comerciales (clusters densos)
centro1 = np.random.randn(150, 2) * 0.3 + np.array([40.4168, -3.7038])  # Madrid centro
centro2 = np.random.randn(100, 2) * 0.4 + np.array([40.4530, -3.6883])  # Zona norte
#centro3 = np.random.randn(80, 2) * 0.35 + np.array([40.3850, -3.7200])  # Zona sur
centro3 = np.random.randn(80, 2) * 0.35 + np.array([40.3850, -3.7500])  # Zona sur

# Ubicaciones residenciales (cluster disperso)
residencial = np.random.randn(120, 2) * 0.8 + np.array([40.4800, -3.6500])

# Transacciones sospechosas (outliers)
anomalias = np.array([
    [40.2, -3.9],    # Muy alejado
    [40.6, -3.4],    # Fuera del patrón
    [40.3, -3.5],
    [40.55, -3.85],
    [40.25, -3.65]
])

# Combinar datos
X_geo = np.vstack([centro1, centro2, centro3, residencial, anomalias])
labels_true = np.array([0]*150 + [1]*100 + [2]*80 + [3]*120 + [-1]*5)

print(f"Dataset: {len(X_geo)} transacciones")
print(f"  - Centro 1 (comercial): 150")
print(f"  - Centro 2 (norte): 100")
print(f"  - Centro 3 (sur): 80")
print(f"  - Residencial: 120")
print(f"  - Anomalías: 5")

In [ ]:
# Visualización del dataset y análisis k-distance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Datos originales
ax = axes[0]
colors = ['#3498db', '#2ecc71', '#9b59b6', '#f39c12', '#e74c3c']
for label in np.unique(labels_true):
    mask = labels_true == label
    if label == -1:
        ax.scatter(X_geo[mask, 0], X_geo[mask, 1], c='red', marker='x',
                   s=100, label='Anomalías', linewidths=2)
    else:
        ax.scatter(X_geo[mask, 0], X_geo[mask, 1], c=colors[label],
                   edgecolors='w', s=40, alpha=0.7)

ax.scatter([], [], c='#3498db', label='Centro comercial 1')
ax.scatter([], [], c='#2ecc71', label='Centro comercial 2')
ax.scatter([], [], c='#9b59b6', label='Centro comercial 3')
ax.scatter([], [], c='#f39c12', label='Residencial')
ax.set_xlabel('Latitud')
ax.set_ylabel('Longitud')
ax.set_title('Ubicaciones de transacciones')
ax.legend(loc='upper left', fontsize=8)

# k-distance graph
ax = axes[1]
k_dist = plot_k_distance(X_geo, k=5, ax=ax)
ax.axhline(y=0.25, color='red', linestyle='--', label='eps sugerido: 0.25')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Aplicar diferentes algoritmos
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# DBSCAN
ax = axes[0, 0]
dbscan_geo = DBSCAN(eps=0.25, min_samples=5)
labels_dbscan = dbscan_geo.fit_predict(X_geo)
n_clusters = len(set(labels_dbscan)) - (1 if -1 in labels_dbscan else 0)
n_ruido = (labels_dbscan == -1).sum()

scatter = ax.scatter(X_geo[:, 0], X_geo[:, 1], c=labels_dbscan, cmap='viridis',
                     edgecolors='w', s=40)
ax.set_title(f'DBSCAN (eps=0.25)\nclusters={n_clusters}, anomalías detectadas={n_ruido}')
ax.set_xlabel('Latitud')
ax.set_ylabel('Longitud')

# HDBSCAN
ax = axes[0, 1]
hdbscan_geo = hdbscan.HDBSCAN(min_cluster_size=20, min_samples=5)
labels_hdbscan = hdbscan_geo.fit_predict(X_geo)
n_clusters = len(set(labels_hdbscan)) - (1 if -1 in labels_hdbscan else 0)
n_ruido = (labels_hdbscan == -1).sum()

scatter = ax.scatter(X_geo[:, 0], X_geo[:, 1], c=labels_hdbscan, cmap='viridis',
                     edgecolors='w', s=40)
ax.set_title(f'HDBSCAN\nclusters={n_clusters}, anomalías detectadas={n_ruido}')
ax.set_xlabel('Latitud')
ax.set_ylabel('Longitud')

# Outlier scores de HDBSCAN
ax = axes[1, 0]
scatter = ax.scatter(X_geo[:, 0], X_geo[:, 1], c=hdbscan_geo.outlier_scores_,
                     cmap='Reds', edgecolors='w', s=40)
plt.colorbar(scatter, ax=ax, label='Outlier Score')
ax.set_title('HDBSCAN - Outlier Scores')
ax.set_xlabel('Latitud')
ax.set_ylabel('Longitud')

# Análisis de anomalías detectadas
ax = axes[1, 1]
# Marcar los puntos detectados como anomalías por HDBSCAN
mask_normal = labels_hdbscan >= 0
mask_anomalia = labels_hdbscan == -1

ax.scatter(X_geo[mask_normal, 0], X_geo[mask_normal, 1], c='#3498db',
           edgecolors='w', s=40, alpha=0.5, label='Normal')
ax.scatter(X_geo[mask_anomalia, 0], X_geo[mask_anomalia, 1], c='#e74c3c',
           marker='X', s=150, edgecolors='w', linewidths=2, label='Anomalía detectada')

# Marcar las anomalías reales
ax.scatter(anomalias[:, 0], anomalias[:, 1], facecolors='none', edgecolors='black',
           s=300, linewidths=2, label='Anomalía real')

ax.set_title('Validación de detección de anomalías')
ax.set_xlabel('Latitud')
ax.set_ylabel('Longitud')
ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Resumen de detección de anomalías
print("Evaluación de detección de anomalías:")
print("=" * 50)

# Anomalías reales
idx_anomalias_reales = set(range(len(X_geo) - 5, len(X_geo)))

# DBSCAN
idx_detectadas_dbscan = set(np.where(labels_dbscan == -1)[0])
tp_dbscan = len(idx_anomalias_reales & idx_detectadas_dbscan)
fp_dbscan = len(idx_detectadas_dbscan - idx_anomalias_reales)
fn_dbscan = len(idx_anomalias_reales - idx_detectadas_dbscan)

print(f"\nDBSCAN (eps=0.25):")
print(f"  Anomalías reales: 5")
print(f"  Detectadas: {len(idx_detectadas_dbscan)}")
print(f"  Verdaderos positivos: {tp_dbscan}")
print(f"  Falsos positivos: {fp_dbscan}")
print(f"  Falsos negativos: {fn_dbscan}")

# HDBSCAN
idx_detectadas_hdbscan = set(np.where(labels_hdbscan == -1)[0])
tp_hdbscan = len(idx_anomalias_reales & idx_detectadas_hdbscan)
fp_hdbscan = len(idx_detectadas_hdbscan - idx_anomalias_reales)
fn_hdbscan = len(idx_anomalias_reales - idx_detectadas_hdbscan)

print(f"\nHDBSCAN:")
print(f"  Anomalías reales: 5")
print(f"  Detectadas: {len(idx_detectadas_hdbscan)}")
print(f"  Verdaderos positivos: {tp_hdbscan}")
print(f"  Falsos positivos: {fp_hdbscan}")
print(f"  Falsos negativos: {fn_hdbscan}")

## 8. Resumen

### Conceptos Clave

1. **Clustering basado en densidad** define clusters como regiones de alta densidad separadas por regiones de baja densidad, permitiendo detectar formas arbitrarias y outliers.

2. **DBSCAN** introduce los conceptos de:
   - **Punto núcleo:** tiene al menos min_samples vecinos en su ε-vecindad.
   - **Punto borde:** no es núcleo pero está en la vecindad de uno.
   - **Ruido:** no es núcleo ni borde.

3. **Selección de parámetros:**
   - El método k-distance graph ayuda a elegir eps.
   - min_samples típicamente se fija en d+1 o 2d (donde d es la dimensionalidad).

4. **OPTICS** genera un ordenamiento que permite extraer clusters a diferentes niveles de densidad mediante el diagrama de alcanzabilidad.

5. **HDBSCAN** combina las ventajas de DBSCAN con clustering jerárquico:
   - No requiere especificar eps.
   - Maneja clusters de densidad variable.
   - Proporciona probabilidades de pertenencia y outlier scores.

### Criterios de Selección

| Situación | Algoritmo Recomendado |
|-----------|-----------------------|
| Clusters de forma arbitraria | DBSCAN, HDBSCAN |
| Detección de outliers | DBSCAN, HDBSCAN |
| Densidad variable | HDBSCAN, OPTICS |
| Sin conocimiento previo de eps | HDBSCAN |
| Análisis exploratorio de estructura | OPTICS |
| Datos geoespaciales | DBSCAN (con índice espacial), HDBSCAN |

---

## Referencias

- Ester, M., Kriegel, H. P., Sander, J., & Xu, X. (1996). A density-based algorithm for discovering clusters in large spatial databases with noise. *KDD*, 96(34), 226-231.
- Ankerst, M., Breunig, M. M., Kriegel, H. P., & Sander, J. (1999). OPTICS: Ordering points to identify the clustering structure. *ACM SIGMOD Record*, 28(2), 49-60.
- Campello, R. J., Moulavi, D., & Sander, J. (2013). Density-based clustering based on hierarchical density estimates. *PAKDD*, 160-172.
- Scikit-learn documentation: https://scikit-learn.org/stable/modules/clustering.html
- HDBSCAN documentation: https://hdbscan.readthedocs.io/

---



# EOF (End Of File)